# Importing Necessary Modules.

In [1]:
import gensim as gs
import pandas as pd

import pyarabic.araby as araby
import pyarabic.number as number

import re

import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os

C:\Users\dell\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Here we are using the Arabic Wiki data Dump 2018

You can find the data on Kaggle.com: https://www.kaggle.com/abedkhooli/arabic-wiki-data-dump-2018

# Extracting Data from Wikipedia XML Dump file.

In [2]:
PATH_WIKI_XML = 'F:\\Hackathon\\easyLearn\\'
FILENAME_WIKI = 'arwiki-20180120-pages-articles-multistream.xml'
FILENAME_ARTICLES = 'articles.csv'
FILENAME_REDIRECT = 'articles_redirect.csv'
FILENAME_TEMPLATE = 'articles_template.csv'
ENCODING = "utf-16"


# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t


pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
pathArticles = os.path.join(PATH_WIKI_XML, FILENAME_ARTICLES)
pathArticlesRedirect = os.path.join(PATH_WIKI_XML, FILENAME_REDIRECT)
pathTemplateRedirect = os.path.join(PATH_WIKI_XML, FILENAME_TEMPLATE)

totalCount = 0
articleCount = 0
redirectCount = 0
templateCount = 0
title = None
text = None
start_time = time.time()

with codecs.open(pathArticles, "w", ENCODING) as articlesFH, \
        codecs.open(pathArticlesRedirect, "w", ENCODING) as redirectFH, \
        codecs.open(pathTemplateRedirect, "w", ENCODING) as templateFH:
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
    redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)
    templateWriter = csv.writer(templateFH, quoting=csv.QUOTE_MINIMAL)

    articlesWriter.writerow(['id', 'title', 'redirect', 'text'])
    redirectWriter.writerow(['id', 'title', 'redirect', 'text'])
    templateWriter.writerow(['id', 'title', 'text'])

    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
        tname = strip_tag_name(elem.tag)

        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = ''
                inrevision = False
                ns = 0
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
        else:
            if tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision:
                id = int(elem.text)
            elif tname == 'redirect':
                redirect = elem.attrib['title']
            elif tname == 'ns':
                ns = int(elem.text)
            elif tname == 'text':
                text = elem.text
            elif tname == 'page':
                totalCount += 1

                if ns == 10:
                    templateCount += 1
                    templateWriter.writerow([id, title, text])
                elif len(redirect) > 0:
                    articleCount += 1
                    articlesWriter.writerow([id, title, redirect, text])
                else:
                    redirectCount += 1
                    redirectWriter.writerow([id, title, redirect, text])

                # if totalCount > 100000:
                #  break

                if totalCount > 1 and (totalCount % 100000) == 0:
                    print("{:,}".format(totalCount))

            elem.clear()

elapsed_time = time.time() - start_time

print("Total pages: {:,}".format(totalCount))
print("Template pages: {:,}".format(templateCount))
print("Article pages: {:,}".format(articleCount))
print("Redirect pages: {:,}".format(redirectCount))
print("Elapsed time: {}".format(hms_string(elapsed_time)))

100,000
200,000
300,000
400,000
500,000
600,000
700,000
800,000
900,000
1,000,000
1,100,000
1,200,000
1,300,000
1,400,000
1,500,000
1,600,000
Total pages: 1,684,890
Template pages: 69,070
Article pages: 508,771
Redirect pages: 1,107,049
Elapsed time: 0:05:10.35


## Loading up the data from the template which we extracted earlier.

In [3]:
wiki_data = "F:\\Hackathon\\easyLearn\\articles_template.csv"

In [4]:
df = pd.read_csv(wiki_data, encoding='utf-16')
df.head()

,id,title,text
0,1323,قالب:Stub,#تحويل [[قالب:بذرة]]\n{{تحويلة قالب من لغات بد...
1,1516,قالب:المهام الحالية,[[ملف:Evolution-tasks.png|يسار]]\n\nفيما يلي ب...
2,1796,قالب:جنوب آسيا,{{شريط\n|اسم = جنوب آسيا\n|عنوان =[[قائمة الدو...
3,1797,قالب:نظام شمسي,{{شريط\n| اسم = نظام شمسي\n| عنوان = {{أمخ}}[...
4,2139,قالب:SelectedArticles,#تحويل [[قالب:مقالة الصفحة الرئيسية المختارة]]...


# Data Cleaning Methods

### The first method we're gonna use is remove_files().
#### It's going to remove everything like [[img.png | ملف ]] or any sort of file in the wikipedia texts.

In [5]:
def remove_files(text):
    result = []
    regex = re.compile("\[.*?\]")
    temp_result = re.findall(regex, text)
    en_letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for string in temp_result:
        contains = any((c in string) for c in en_letters)
        if contains:
            result.append(string)
    for el in result:
        text = text.replace(el,' ')
    return text

#### This method cleans the text from some certain characters and replace them with the appropriate characters.

In [6]:
def clean_some_chars(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!'
              ,"[","]","{","}","*",":","#","$","€","£","~","<",">","/","|","'",",",'=','(',')','+','•',';','&','–','♦','%'
             ,'»','»','·']
    
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ',' ',' '
               ," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," "," ",' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
              ,' ',' ',' ']
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    text = text.replace('\'','')
    # removing numbers
    text = ''.join([i for i in text if not i.isdigit()])
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

#### The next method removes all English letters from the text.

In [7]:
def clean_english_chars(text):
    search = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z'
             ,'A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Z','Y','Z']
    for i in range(0, len(search)):
        text = text.replace(search[i],' ')
    return text

#### This method removes all unnecessary whitespaces from the text.

That includes whitespaces from the beginning of the text, the end of it and all unwanted onces underlying in between

In [8]:
def remove_unnecessary_spaces(text):
    return re.sub(' +',' ',text)

#### The coming method removes all non-arabic letters from the text.

The text might have some foreign languages letters (or characters) that are unwanted in our data which is meant to be pure Arabic text.

A sample of such a text might be something like this: 

"ل اكثر من مقال انجليزيه اكثر من مقال المانيه اسبانيه ñ فرنسيه ç ايطاليه يابانيه 日本語 هولنديه بولنديه برتغاليه ê روسيه Русский سويديه اكثر من مقال قطلونيه à تشيكيه Č š دنماركيه اسبرانتو مجريه نرويجيه بوكمال å رومانيه â ă سلوفاكيه č فنلنديه سويديه تركيه ü ç اوكرانيه Українська صينيه مبسطه 中文 اكثر من مقال اندونيسيه بلغاريه Български استونيه فارسيه فارسی عبريه עברית كوريه 한국어 ليتوانيه ų انكليزيه بسيطه سلوفينيه šč صربيه Српски اردو اردو فيتناميه ế ệ اكثر من مقال ماليزيه بشنوبريا مانيبوري ইমার ঠারবিষ্ণুপ্রিয়া মণিপুরী بريتانيه بوسنيه يونانيه Ελληνικά باسكيه غاليكيه هنديه हिन्दी كرواتيه ايسلنديه Í جيورجيه ქართული لاتينيه لوكسمبورغيه"

In [9]:
def remove_non_arabic_letters(text):
    
    '''
    ALEF_MADDA       = u'\u0622' 
    ALEF_HAMZA_ABOVE = u'\u0623' 
    WAW_HAMZA        = u'\u0624' 
    ALEF_HAMZA_BELOW = u'\u0625' 
    YEH_HAMZA        = u'\u0626' 
    ALEF             = u'\u0627' 
    BEH              = u'\u0628' 
    TEH_MARBUTA      = u'\u0629' 
    TEH              = u'\u062a' 
    THEH             = u'\u062b' 
    JEEM             = u'\u062c' 
    HAH              = u'\u062d' 
    KHAH             = u'\u062e' 
    DAL              = u'\u062f' 
    THAL             = u'\u0630' 
    REH              = u'\u0631' 
    ZAIN             = u'\u0632' 
    SEEN             = u'\u0633' 
    SHEEN            = u'\u0634' 
    SAD              = u'\u0635' 
    DAD              = u'\u0636' 
    TAH              = u'\u0637' 
    ZAH              = u'\u0638' 
    AIN              = u'\u0639' 
    GHAIN            = u'\u063a' 
    TATWEEL          = u'\u0640' 
    FEH              = u'\u0641' 
    QAF              = u'\u0642' 
    KAF              = u'\u0643' 
    LAM              = u'\u0644' 
    MEEM             = u'\u0645' 
    NOON             = u'\u0646' 
    HEH              = u'\u0647' 
    WAW              = u'\u0648' 
    ALEF_MAKSURA     = u'\u0649' 
    YEH              = u'\u064a' 
    MADDA_ABOVE      = u'\u0653' 
    HAMZA_ABOVE      = u'\u0654' 
    HAMZA_BELOW      = u'\u0655' 
    LAM_ALEF                     = u'\ufefb' 
    LAM_ALEF_HAMZA_ABOVE         = u'\ufef7' 
    LAM_ALEF_HAMZA_BELOW         = u'\ufef9' 
    LAM_ALEF_MADDA_ABOVE         = u'\ufef5' 
    '''
    
    regex = re.compile(r'[\u0622\u0623\u0624\u0625\u0626\u0627\u0628\u0629\u062a\u062b\u062c\u062d\u062e\u062f\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063a\u0640\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u0649\u064a\u0653\u0654\u0655\ufefb\ufef7\ufef9\ufef5]')
    # removing Arabic letters from the text and storing the result in the varialbe: unwanted_str .
    unwanted_str = regex.sub(' ',text)
    # Creating a list containing all of the unwanted characters, letters and symbols.
    unwanted_list_of_strs = list(unwanted_str.replace(" ", ""))
    # Cleaning the unwanted list of characters out of the text
    for i in range(0, len(unwanted_list_of_strs)):
        text = text.replace(unwanted_list_of_strs[i], " ")
    
    text = remove_unnecessary_spaces(text)
    
    return text

### The last method is to remove a single letter that has no meaning in the context.

In [17]:
def concatenate_list_into_string(lis_strs):
    result = ""
    for el in lis_strs:
        result += " " + el
    return result

In [18]:
def remove_single_letters(text):
    words = text.split(' ')
    waw = 'و'
    for word in words:
        if len(word.strip()) == 1:
            if word != waw:
                words.remove(word)
    text = concatenate_list_into_string(words)
    return text

### Getting all of our cleaning methods under one big general method.

In [10]:
def clean_text(text):
    # removing files
    text = remove_files(text)
    # removing some unuseful chars
    text = clean_some_chars(text)
    # removing english chars
    text = clean_english_chars(text)
    # removing tashkeel
    text = araby.strip_tashkeel(text)
    # removing longation
    text = araby.strip_tatweel(text)
    # removing unwanted spaces
    text = remove_unnecessary_spaces(text)
    # removing non-arabic characters
    text = remove_non_arabic_letters(text)
    # removing single unwanted letters
    text = remove_single_letters(text)
    # returning result
    return text    

### Now, let's prepare our data to be cleaned.

#### Getting rid of Nan values.

In [11]:
df.dropna()

,id,title,text
0,1323,قالب:Stub,#تحويل [[قالب:بذرة]]\n{{تحويلة قالب من لغات بد...
1,1516,قالب:المهام الحالية,[[ملف:Evolution-tasks.png|يسار]]\n\nفيما يلي ب...
2,1796,قالب:جنوب آسيا,{{شريط\n|اسم = جنوب آسيا\n|عنوان =[[قائمة الدو...
3,1797,قالب:نظام شمسي,{{شريط\n| اسم = نظام شمسي\n| عنوان = {{أمخ}}[...
4,2139,قالب:SelectedArticles,#تحويل [[قالب:مقالة الصفحة الرئيسية المختارة]]...
5,2178,قالب:تقويم يناير,{{تقويم شهري بدون عام|شهر=1|سنة={{CURRENTYEAR}...
6,2179,قالب:تقويم فبراير,{{تقويم شهري بدون عام|شهر=2|سنة={{CURRENTYEAR}...
7,2183,قالب:رؤساء الحكومة الإسرائيلية,{{شريط\n|اسم = رؤساء الحكومة الإسرائيلية\n|عنو...
8,2319,قالب:تقويم مايو,{{تقويم شهري بدون عام|شهر=5|سنة={{CURRENTYEAR}...
9,2373,قالب:تقويم يونيو,{{تقويم شهري بدون عام|شهر=6|سنة={{CURRENTYEAR}...


### Making sure that all of the data we have under the text column in our dataframe are string typed.

In [12]:
df['text'] = df['text'].astype(str)

Taking a look at the data.

In [13]:
df['text']

0        #تحويل [[قالب:بذرة]]\n{{تحويلة قالب من لغات بد...
1        [[ملف:Evolution-tasks.png|يسار]]\n\nفيما يلي ب...
2        {{شريط\n|اسم = جنوب آسيا\n|عنوان =[[قائمة الدو...
3        {{شريط\n| اسم =  نظام شمسي\n| عنوان = {{أمخ}}[...
4        #تحويل [[قالب:مقالة الصفحة الرئيسية المختارة]]...
5        {{تقويم شهري بدون عام|شهر=1|سنة={{CURRENTYEAR}...
6        {{تقويم شهري بدون عام|شهر=2|سنة={{CURRENTYEAR}...
7        {{شريط\n|اسم = رؤساء الحكومة الإسرائيلية\n|عنو...
8        {{تقويم شهري بدون عام|شهر=5|سنة={{CURRENTYEAR}...
9        {{تقويم شهري بدون عام|شهر=6|سنة={{CURRENTYEAR}...
10       {{تقويم شهري بدون عام|شهر=4|سنة={{CURRENTYEAR}...
11       {{تقويم شهري بدون عام|شهر=7|سنة={{CURRENTYEAR}...
12       {{تقويم شهري بدون عام|شهر=8|سنة={{CURRENTYEAR}...
13       <div style="clear:left;">\n<table style="float...
14       {{تقويم شهري بدون عام|شهر=11|سنة={{CURRENTYEAR...
15       {{تقويم شهري بدون عام|شهر=10|سنة={{CURRENTYEAR...
16       {{تقويم شهري بدون عام|شهر=9|سنة={{CURRENTYEAR}.

### Before Cleaning all the text we have. Let's try our cleaning only the second article and see our results.

In [14]:
second_article = df['text'].loc[1]

Here is the text of the second article before cleaning it:

In [15]:
second_article

"[[ملف:Evolution-tasks.png|يسار]]\n\nفيما يلي بعض المهام الحالية التي قد تساهم بها:\n* '''تنمية مقالات البذرة''': اختر المقالة الذي تراها مناسبة لك والذي تنوي تنميتها وتحسينها من القوائم التالية:\n** [[:تصنيف:بذرة|بذرة]]\n** [[:تصنيف:بذرة رياضيات]]\n** [[:تصنيف:بذرة علوم|بذرة علوم]]\n** [[:تصنيف:بذرة تاريخ|بذرة تاريخ]]\n** [[:تصنيف:بذرة جغرافيا|بذرة جغرافيا]]\n** [[:تصنيف:بذرة أعلام|بذرة أعلام]]\n** [[:تصنيف:بذرة رياضة|بذرة رياضة]]\n** [[:تصنيف:بذرة ثقافة]]\n** [[:تصنيف:بذرة سياسة]]\n** [[:تصنيف:بذرة فلسفة]]\n* '''إنشاء صفحات جديدة''': من الممكن أن تساهم بذلك عن طريق إنشاء مقالة عن أي من المواضيع المذكورة في صفحة [[ويكيبيديا:مواضيع مقترحة|المواضيع المقترحة]].\n* '''إنشاء صفحات تحويل''': يمكنك أيضا المشاركة في تحسين الموسوعة عن طريق إنشاء صفحات تحويل بعناوين شائعة لكنها لا تخضع [[ويكيبيديا:تسمية المواضيع|لسياسة تسمية المقالات]] في ويكيبيديا عن طريق إضافة:\n\n<code><nowiki>#تحويل [[عنوان المقال المراد التحويل إليه]]</nowiki></code>\n* '''تنقيح المقالات الموجودة''': ابحث عن المقال الذي تر

Let's apply the magic of our cleaning method.

Our approach to clean the arabic text data seems to be working properly.

### Trying to clean the rest of the data.

In [19]:
df['text'] = df['text'].apply(clean_text)

Let's take a look at the results and keep our fingers crossed hoping to succeed!

In [20]:
df.head(20)

,id,title,text
0,1323,قالب:Stub,تحويل قالب بذره تحويله قالب من لغات بديله
1,1516,قالب:المهام الحالية,فيما يلي بعض المهام الحاليه التي قد تساهم بها...
2,1796,قالب:جنوب آسيا,شريط اسم جنوب اسيا عنوان قائمه الدول دول ومقا...
3,1797,قالب:نظام شمسي,شريط اسم نظام شمسي عنوان امخ المجموعه الشمسيه...
4,2139,قالب:SelectedArticles,تحويل قالب مقاله الصفحه الرئيسيه المختاره تحو...
5,2178,قالب:تقويم يناير,تقويم شهري بدون عام شهر سنه تصنيف قوالب تقويم...
6,2179,قالب:تقويم فبراير,تقويم شهري بدون عام شهر سنه تصنيف قوالب تقويم...
7,2183,قالب:رؤساء الحكومة الإسرائيلية,شريط اسم رؤسا الحكومه الاسرائيليه عنوان رئيس ...
8,2319,قالب:تقويم مايو,تقويم شهري بدون عام شهر سنه تصنيف قوالب تقويم...
9,2373,قالب:تقويم يونيو,تقويم شهري بدون عام شهر سنه تصنيف قوالب تقويم...


## Everything looks just fine.

### Now, we can move on to the next steps.